## Mapping Great Circle Arcs

The shortest path between two points on a sphere is the minor arc of a **great circle** that passes through both points and the center of the sphere. Great circle arcs are also called **geodesic lines**. This notebook explores creating and mapping [great circle](https://en.wikipedia.org/wiki/Great_circle) arcs - follow that link to read about them on Wikipedia.

In [ ]:
# Import the python libraries we weill use
from datascience import *
import folium
import numpy as np
import pyproj

# Simple function to show folium maps inline
from IPython.display import HTML

def inline_map(m, height=500):
    """Takes a folium instance and embed HTML."""
    m._build_map()
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{0}" '
                 'style="width: 100%; height: {1}px; '
                 'border: none"></iframe>'.format(srcdoc, height))
    return embed

## Plotting a flight path

Let's assume airplanes fly along great circle paths because the want the shortest route. We can identify the points for two airports, Seattle and London, and then plot the line between them on a map.

In [ ]:
# Identify the coordinates for two locations - Seattle and London 
Seattle = (47.606, -122.33)
London = (51.5, 0.0)


In [ ]:
# Create a map using folium
ctr_lon = np.mean([Seattle[1], London[1]]) # calculate the center of longitude values
ctr_lat = np.mean([Seattle[0], London[0]]) # center of lat values

m = folium.Map([ctr_lat, ctr_lon], zoom_start=2) # set the map center and zoom level

# Add the line to the map
m.line(locations=[Seattle, London], popup="Straight line distance on plane", line_color='blue', line_opacity=1.0)

inline_map(m)


Now that line doesn't look like the kind of flight path you would see illustrated in an airline magazine. That's because with only two points the map cannot create a curved line on a flat map. We need to add some points along the line of a [great circle](https://en.wikipedia.org/wiki/Great_circle).  One way to do this is with the **pyproj** library.  Read the comments below to see how this is done. Also refer to the [pyproj](http://jswhit.github.io/pyproj/) online documentation to read abou the specific functions used below.

In [ ]:
# Calculate great circle arcs between points

g = pyproj.Geod(ellps='WGS84')  # Create a WGS84 CRS object

startlong = Seattle[1]  
startlat = Seattle[0]  
endlong = London[1]  
endlat = London[0]  

# Use the g.inv function to compute the bearing to and from each point as well as the distance between the points
(bearing_pt1_to_pt2, bearing_pt2_to_pt1, distance_pt1_to_pt2) = g.inv(startlong, startlat, endlong, endlat)

# Use the g.npts function to calculate a line string along the path between these points adding points every 10000 km
lonlats = g.npts(startlong, startlat, endlong, endlat,
                 1 + int(distance_pt1_to_pt2 / 10000))

# g.npts doesn't include start/end points, so prepend/append them
lonlats.insert(0, (startlong, startlat))
lonlats.append((endlong, endlat))

# Print the output from these functions
print('Distance in km: ', distance_pt1_to_pt2 / 1000)
print('Bearing_pt1_to_pt2: ', bearing_pt1_to_pt2)
print('Bearing_pt2_to_pt1: ', bearing_pt2_to_pt1)

Take a look at the coordinates returned by **g.npts**. They are in lonlat order but the **folium** mapping library needs them in latlon order. We can use a simple numpy function **fliplr** to reverse the order.


In [ ]:
print(lonlats)

In [ ]:
# flip the order of the lonlats to latlon

latlons = np.fliplr(lonlats)
print(len(latlons))
print(latlons)


### Plotting Great Circles

Now that we have the points for the curved line of a great circle arc we can plot it on our map.  This is the flight path that a plane would take if it wanted to fly along the shortest route.

In [ ]:
m = folium.Map([ctr_lat, ctr_lon], zoom_start=2)

m.line(locations=latlons,popup = ('Geodesic line on sphere'), line_color='red', line_opacity=1.0)

m.line(locations=[Seattle, London], popup="Straight line distance on plane", line_color='blue', line_opacity=1.0)

inline_map(m)

## Question 1
Why do we need to compute the points along the great circle arc using the WGS84 projection? What does *geodesic* mean?

### Double-click  here to input your answer to Question 1 in this cell.





# Part 2 - Got to Get Away
Let's expand on this work and create a flight path map for all flights from San Francisco International Airport (SFO). We will load a table of airport locations and then one of airport routes.  These data are from http://openflights.org

In [ ]:
airports = Table.read_table("./airports.csv")

In [ ]:
airports


Let's remove some unneeded columns from the airports table to make it more readable.

In [ ]:
# Subset the airports table 
airport_locs = airports.select(['code_iata','latitude','longitude','country'])

airport_locs


Now we will load in a table of routes. This table listes the airline, start airport and end, or destination, airport as well as other data about these routes.

In [ ]:
routes = Table.read_table("./routes.csv")

In [ ]:
routes


Let's subset the routes data to select only routes where the start airport is SFO and the codeshare value does not equal yes (y). Code sharing really complicates this as multiple airlines can claim the same flight.

In [ ]:
sfo_departures = routes.where(routes['airport_st'] == 'SFO')
sfo_departures = sfo_departures.where(sfo_departures['codeshare'] != 'Y')

In [ ]:
sfo_departures

We can retrieve and store the latitude and longitude coordinates for SFO because we only have one departure airline. But, we need to add the coordinates for all the destination airports so that we can map the routes. We can do that by joining the **airport_locs** table to the **sfo_departures** table.

In [ ]:
# Get the lat,lon coordinates for SFO
sfo_lon = airports.where(airports['code_iata']=='SFO')['longitude'][0]
sfo_lat = airports.where(airports['code_iata']=='SFO')['latitude'][0]
print("SFO: %.4f latitude, %.4f longitude" % (sfo_lat, sfo_lon) )

In [ ]:
# Add the destination coordinates to the sfo_departures table.
# Then rename the columns so we can remember what the values mean.
sfo_departures = sfo_departures.join('airport_end',airport_locs,'code_iata')
sfo_departures.relabel('latitude', 'dest_lat')
sfo_departures.relabel('longitude', 'dest_lon')
sfo_departures.relabel('country', 'dest_country')

### No Passport, No Problem
Let's limit the geographic extent of our destinations and only select destination airports in the United States.

In [ ]:
sfo_departures_usa = sfo_departures.where('dest_country','United States')
sfo_departures_usa


## Question 2
Use the next two code cells to complete the **addRouteToMap** function. Then use it to add each route in the **sfo_departures_usa** table to a folium map.  Use the **pyproj** code to calculate great circle arcs to create the lines.

In [ ]:
#def addRouteToMap(the_map, start_airport, dest_airport):

    # Add Your code below
    
    # First get the coords for the start and dest airports
    
    # Second, Calculate great circle arcs between points
    
    # Third, reverse the order of the coordinates
    
    # Then, add the coordinates for this route to the map
    ## the_map.line(....)
    



In [ ]:
# Map the Routes
# Add your code to this cell - uncomment lines as needed

# First calculate the map center coordinates
## ctr_lat =  
## ctr_lon =  

# Create the map
## m = folium.Map(...) 

# Add the lines to the map
## sfo_departures_usa.apply(lambda x, y: addRouteToMap(m, x, y) , ['airport_st', 'airport_end'])

# Display the map
## inline_map(m)

## Question 3 - Optional. Mapping all Routes for one Airline
### *This is an extra credit question - up to 5 pts extra credit.*

Build off the the code above to:
- Select all routes served by United Airlines (IATA code UA) to create a ua_routes table.
- Add start_lat, start_lon and start_country to the ua_routes table.
- Add end_lat, end_lot, and end_country to the ua_routes table.
- Remove any routes that are codeshare = Y and any routes that start or end outside of the United States.
- Map the routes - centering the map on the center lat and lon of all routes.

Show your work in the cells below. Uncomment and add code as needed.

In [ ]:
# Get the United Airlines (US) Routes
## ua_routes = ...

# Join with airport_locs to add the destination airport coordinates
##ua_routes = ...
##ua_routes.relabel(...)
##ua_routes.relabel(...)
##ua_routes.relabel(...)

# Join with airport_locs to add the TAKE-OFF airport coordinates
##ua_routes = ...
##ua_routes.relabel(...)
##ua_routes.relabel(...)
##ua_routes.relabel(...)

# Limit to USA origins and destinations
##ua_routes_usa = ua_routes.where(np.logical_and(...))

# Remove code sharing routes
##ua_routes_usa = ua_routes_usa.where(...)
##ua_routes_usa

In [ ]:
# Function to map all routes for an airline
##def addRouteToMap2(the_map, start, dest):
    
    # YOUR CODE HERE
    
    
    ##the_map.line(...)
    

In [ ]:
# Map the routes

# Get the map center coordinates
##ctr_lat = ...
##ctr_lon = ...

# Create the map
##m = folium.Map(...) 

# Map the routes
##ua_routes_usa.apply(...)
 
# Show the map
##inline_map(m)

## Question 4

Let's explore a route that is outside of the United States. Using the simple method above for mapping the great circle path between Seattle and London, create a map of the great circle path between the SFO and the Hong Kong International Airport (HKG). You can get the HKG coordinates from the airports table. What problem do you see with this route? Is this route the minor arc of a great circle? If not what is it and what is causing the problem? Show your code for mapping the route and your discussion in the two cells below.


In [ ]:
# Add your code for Question 4 to this cell.

## Double-click on this Markdown cell to add your comments for Question 4.

## What to submit via bCourses

Submit a zip file containing the following:
- your completed notebook as an *.ipynb document
- your notebook as a PDF file


### Due Date: Monday, March 14, 2016 at 4pm.

### Acknowledgements
This notebook is based on http://blog.cartodb.com/jets-and-datelines.